In [1]:
import sys
sys.path.append('/home/simon/git/lda/code/')

In [2]:
from ms2lda_feature_extraction import LoadMGF

In [3]:
mgf_file = '/home/simon/Dropbox/MolecularNetworking_GCF-MF_Crusemann_Duncan/Crusemann_only_Clutered_Data/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'

In [4]:
min_ms2_intensity = 25

In [5]:
l = LoadMGF(min_ms2_intensity = 25)
ms1,ms2,metadata = l.load_spectra([mgf_file])

Filtering MS2 on intensity
85977 MS2 remaining


In [6]:
%load_ext autoreload
%autoreload 2

In [56]:
mols_as_dicts = {}
for m in ms1:
    mols_as_dicts[m.name] = []

In [57]:
for f in ms2:
    m = f[3]
    mols_as_dicts[m.name].append((f[0],f[2]))

In [58]:
for m in mols_as_dicts:
    mols_as_dicts[m] = sorted(mols_as_dicts[m],key = lambda x: x[0])

In [59]:
for m in mols_as_dicts:
    max_val = max([x[1] for x in mols_as_dicts[m]])
    new_spec = []
    for mz,i in mols_as_dicts[m]:
        new_spec.append((mz,100.0*i/max_val))
    mols_as_dicts[m] = new_spec

In [60]:
parentmass = {}
for m in ms1:
    parentmass[m.name] = metadata[m.name].get('parentmass')

Compute all the scores

In [61]:
scores = []
n_mols = len(ms1)
import numpy as np


In [120]:
from spectrum_alignment import score_alignment
tol = 0.5
n_done = 0
max_align = 0
max_score = 0
for i,m1 in enumerate(ms1[:-1]):
    for j,m2 in enumerate(ms1[i+1:]):
        score,alignment = score_alignment(mols_as_dicts[m1.name],mols_as_dicts[m2.name],parentmass[m1.name],parentmass[m2.name],tol)
#         scoremat[i,j] = score
#         n_matches[i,j] = len(alignment)
        if len(alignment) > 0:
            scores.append((m1.name,m2.name,score,len(alignment)))
            if len(alignment) > max_align:
                max_align = len(alignment)
#                 print max_align
            if score > max_score:
                max_score = score
#                 print max_score
    n_done += 1
    if n_done % 100 == 0:
        print n_done,n_mols
#         scores.append((m1,m2,score))
    


100 5930
200 5930
300 5930
400 5930
500 5930
600 5930
700 5930
800 5930
900 5930
1000 5930
1100 5930
1200 5930
1300 5930
1400 5930
1500 5930
1600 5930
1700 5930
1800 5930
1900 5930
2000 5930
2100 5930
2200 5930
2300 5930
2400 5930
2500 5930
2600 5930
2700 5930
2800 5930
2900 5930
3000 5930
3100 5930
3200 5930
3300 5930
3400 5930
3500 5930
3600 5930
3700 5930
3800 5930
3900 5930
4000 5930
4100 5930
4200 5930
4300 5930
4400 5930
4500 5930
4600 5930
4700 5930
4800 5930
4900 5930
5000 5930
5100 5930
5200 5930
5300 5930
5400 5930
5500 5930
5600 5930
5700 5930
5800 5930
5900 5930


In [121]:
with open('crusemann_scores_05.csv','w') as f:
    writer = csv.writer(f)
    for row in scores:
        writer.writerow(row)

In [111]:
from mol_networks import create_edge_dict,Network
min_frag_overlap = 4
min_score = 0.5
max_component_size = 250
edge_dict = create_edge_dict(scores,min_frag_overlap = min_frag_overlap,min_score = min_score)
network = Network(edge_dict)
man_network = network.copy()
man_network.top_k_filter(k=10)
components = man_network.convert_to_components(max_component_size=max_component_size,min_score = min_score)

Found 1 components
Found 2 components
Found 3 components
Found 4 components
Found 5 components
Found 6 components
Found 7 components
Found 8 components
Found 9 components
Found 10 components
Found 11 components
Found 12 components
Found 13 components
Found 14 components
Found 15 components
Found 16 components
Found 17 components
Found 18 components
Found 19 components
Found 20 components
Found 21 components
Found 22 components
Found 23 components
Found 24 components
Found 25 components
Found 26 components
Found 27 components
Found 28 components
Found 29 components
Found 30 components
Found 31 components
Found 32 components
Found 33 components
Found 34 components
Found 35 components
Found 36 components
Found 37 components
Found 38 components
Found 39 components
Found 40 components
Found 41 components
Found 42 components
Found 43 components
Found 44 components
Found 45 components
Found 46 components
Found 47 components
Found 48 components
Found 49 components
Found 50 components
Found 51 

Cropping components to have max size = 250
	978 components ok, 3 need cropping
Cropping components to have max size = 250
	981 components ok, 3 need cropping
Cropping components to have max size = 250
	984 components ok, 3 need cropping
Cropping components to have max size = 250
	987 components ok, 3 need cropping
Cropping components to have max size = 250
	990 components ok, 3 need cropping
Cropping components to have max size = 250
	993 components ok, 3 need cropping
Cropping components to have max size = 250
	996 components ok, 3 need cropping
Cropping components to have max size = 250
	999 components ok, 3 need cropping
Cropping components to have max size = 250
	1002 components ok, 3 need cropping
Cropping components to have max size = 250
	1005 components ok, 3 need cropping
Cropping components to have max size = 250
	1008 components ok, 3 need cropping
Cropping components to have max size = 250
	1011 components ok, 3 need cropping
Cropping components to have max size = 250
	1014

In [112]:
print "There are {} components".format(len(components))
print
print
for component in components:
    print
    print component

There are 1232 components



Network has 15 nodes
document_5823 (degree = 10)	(document_5825,0.887400790207) (document_5350,0.790537304112) (document_5767,0.655132735562) (document_5766,0.714436449606) (document_5765,0.779807724314) (document_5798,0.786660686729) (document_5797,0.875255456422) (document_5800,0.617896130088) (document_5801,0.731098877277) (document_5499,0.905581553546)
document_5894 (degree = 1)	(document_5781,0.622542728703)
document_5781 (degree = 4)	(document_5499,0.640635431651) (document_5894,0.622542728703) (document_5500,0.512368031822) (document_5825,0.599309365772)
document_5797 (degree = 10)	(document_5823,0.875255456422) (document_5825,0.79231599437) (document_5350,0.770124018795) (document_5799,0.650237617279) (document_5767,0.693060729107) (document_5766,0.784112807431) (document_5765,0.851778122725) (document_5798,0.721337400575) (document_5801,0.720154253827) (document_5499,0.831704445011)
document_5798 (degree = 10)	(document_5823,0.786660686729) (docume

In [113]:
import csv
with open('crusemann_edges.csv','w') as f:
    writer = csv.writer(f)
    network.write_network(writer,heads = True)

In [114]:
with open('crusemann_components.csv','w') as f:
    writer = csv.writer(f)
    for i,component in enumerate(components):
        if i==0:
            heads = True
        else:
            heads = False
        component.write_network(writer,heads = heads)

In [115]:
component_sizes = {}
for component in components:
    size = component.n_nodes()
    if not size in component_sizes:
        component_sizes[size] = 1
    else:
        component_sizes[size] += 1
cs = zip(component_sizes.keys(),component_sizes.values())
cs = sorted(cs,key = lambda x: x[0])
for s,c in cs:
    print s,c
print len(components)-component_sizes[1]

1 814
2 191
3 66
4 42
5 18
6 13
7 8
8 7
9 7
10 1
11 2
12 5
13 6
14 3
15 6
16 1
17 1
18 2
19 2
20 5
21 1
23 2
24 2
25 1
26 2
27 1
28 1
29 1
30 1
32 3
33 1
34 1
35 1
38 1
43 1
49 1
51 1
53 2
61 1
70 1
76 1
104 1
188 1
249 2
250 1
418


Grab some data from the server

In [88]:
import requests
crusemann_id = 583
crusemann_mzdiff_id = 582
url = 'http://ms2lda.org/basicviz/get_all_doc_data/{}'.format(crusemann_id)
response = requests.get(url)


In [102]:
# make a dictionary of doc names to motifs
doc_motifs = {}
motif_docs = {}
for doc_name,_,motifs in response.json():
    doc_motifs[doc_name] = {}
    for motif_name,p,o in motifs:
        doc_motifs[doc_name][motif_name] = (p,o)
        if not motif_name in motif_docs:
            motif_docs[motif_name] = {}
        motif_docs[motif_name][doc_name] = (p,o)

In [118]:
print motif_docs['motif_24']
doc_set = set(motif_docs['motif_24'].keys())
print doc_set

{u'document_4619': (0.0198345629767357, 0.0816474035301373), u'document_231': (0.878934386174287, 0.0294346142772308), u'document_4575': (0.0230404701308814, 0.184359899108348), u'document_379': (0.0271346316759214, 0.0413725721216124), u'document_4180': (0.175198353809687, 0.0881036493495618), u'document_5313': (0.0222237492855581, 0.0430490175694649), u'document_4128': (0.0411489607385251, 0.0193940439650437), u'document_5335': (0.406814354163913, 0.0485022178882047), u'document_822': (0.124311025173957, 0.04145268784811), u'document_2984': (0.45645277743345, 0.232024047896673), u'document_4235': (0.269218681107128, 0.0909874174149197), u'document_4339': (0.0726479639748166, 0.233415192675697), u'document_2983': (0.402094703857634, 0.230675337718747), u'document_4258': (0.257957122177343, 0.0843458699555986), u'document_2074': (0.0447447550465354, 0.0414323307307675), u'document_2490': (0.854419774797443, 0.103779701085604), u'document_3121': (0.0289142218318227, 0.184379188362364), 

In [119]:
with open('crusemann_motif_24.csv','w') as f:
    writer = csv.writer(f)
    with open('crusemann_components.csv','r') as g:
        reader = csv.reader(g)
        writer.writerow(reader.next() + ['motif_24']) # headers
        for line in reader:
            doc1 = line[0]
            doc2 = line[1]
            score = line[2]
            if doc1 in doc_set and doc2 in doc_set:
                m4 = 1
            else:
                m4 = 0
            writer.writerow([doc1,doc2,score,m4])